## Agenda
### Part 0: Basic Data Cleaning
The first step is to do some basic data cleaning and rid of all the columns that won't be of any use acrross any of the projects going forward, and add some useful columns to the dataset based on the existing ones that will come handy in both Data Analysis and ML/NLP.

* **Drop:**
['Name', 'AuthorName', 'CookTime', 'PrepTime', 'TotalTime', 'DatePublished', 'Description', 'Images', 'ReviewCount', 'RecipeServings', 'RecipeYield', 'RecipeInstructions', 'url']

* **Add:**
['TotalMinutes', 'YearPublished', 'MonthPublished', 'DayPublished', 'HourPublished']

* **Replace:**
['RecipeIngredientQuantities', 'RecipeIngredientParts'] with ones scraped from food.com froms scratch.

**Save:**
BasicCleanData.pkl

We can perform classical data analysis on BasicCleanData.pkl


#### Imports and sanity checks

In [1]:
import sys
sys.executable

'C:\\Users\\mathe\\anaconda3\\envs\\deepchef\\python.exe'

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [6]:
recipes = pd.read_parquet('C:/Users/mathe/Dropbox/++Tech/++Projects/DeepChef/recipes_with_urls.parquet')

In [8]:
recipes.sample(5)

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions,url
5188,8116.0,Finnish Curd Cake,39547,Julesong,None,PT0S,PT1H,2000-03-13 21:48:00+00:00,Make and share this Finnish Curd Cake recipe f...,[],...,95.0,366.1,33.4,1.1,14.0,7.9,8.0,None,"[Butter the baking dish, pie pan, or springfor...",https://www.food.com/recipe/Finnish-Curd-Cake-...
12893,16141.0,"Danish ""Sillsallet"" (Herring Salad)",4470,Bergy,PT10M,PT20M,PT30M,2001-12-22 09:17:00+00:00,Traditional part of a Danish smorgasbord. Alwa...,[https://img.sndimg.com/food/image/upload/w_55...,...,0.0,321.0,28.5,4.8,14.4,3.4,NaN,None,"[Chop the herring, potatoe, beets and apple in...","https://www.food.com/recipe/Danish-""Sillsallet..."
158388,165926.0,Chicken Lasagna,281399,Calee,PT45M,PT45M,PT1H30M,2006-04-26 17:37:00+00:00,Hollandaise sauce and asparagus tips makes thi...,[https://img.sndimg.com/food/image/upload/w_55...,...,61.5,432.3,27.3,2.1,2.3,25.9,8.0,None,"[Saute mushrooms and onion until soft., Spread...",https://www.food.com/recipe/Chicken-Lasagna-16...
510479,528989.0,Sourdough Rolls,1803541378,Late Night Gourmet,None,PT96H,PT96H,2016-11-10 19:30:00+00:00,Make and share this Sourdough Rolls recipe fro...,[https://img.sndimg.com/food/image/upload/w_55...,...,0.0,292.0,19.4,1.1,1.5,2.6,24.0,None,[Soak 1 teaspoon dried starter in 1 tablespoon...,https://www.food.com/recipe/Sourdough-Rolls-52...
444323,460708.0,Raspberry and Lychee Sorbet (Thermomix),214855,Wendys Kitchen,PT5M,PT5M,PT10M,2011-07-22 04:53:00+00:00,Make and share this Raspberry and Lychee Sorbe...,[],...,0.0,12.3,26.3,1.2,25.1,1.0,NaN,None,[Mill the sugar on Speed 9 for 6 secs til you ...,https://www.food.com/recipe/Raspberry-and-Lych...
